Note: During this assignment I faced a problem with converting my model outputs to accuracy and f1 score. Therefore, I utilized the loss function to evaluate the model performance.

Joao Atz Dick

Student ID : 1009786288

Sharing link:

https://colab.research.google.com/drive/1h14w-8i2sVW1Xjzdp5gTd90kRuQe_ev3?usp=sharing

### 1-4) Data Preparation

In [122]:
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.0 MB/s eta 0:00:00


In [125]:
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from datetime import datetime
from torcheval.metrics import BinaryF1Score

In [5]:
#change path to your dataset location
df = pd.read_csv('/content/drive/MyDrive/ECE1513/indian_liver_patient.csv').fillna(0)
df.Gender = pd.factorize(df.Gender)[0]
df_norm = (df-df.min())/(df.max()-df.min())


In [77]:
if torch.cuda.is_available():
 device = "cuda:0"
else:
 device = "cpu"

# change 'random_state' for different random seed
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df.iloc[:,:-1], df_norm.iloc[:,-1:], test_size= 0.3, random_state=1)
X_train_norm, X_test_norm, y_train_norm, y_test_norm = sklearn.model_selection.train_test_split(df_norm.iloc[:,:-1], df_norm.iloc[:,-1:], test_size= 0.3, random_state=1)

#df to tensor
X_train = torch.Tensor(np.array(X_train)).to(device)
y_train = torch.Tensor(np.array(y_train)).type(torch.float).to(device)
X_test = torch.Tensor(np.array(X_test)).to(device)
y_test = torch.Tensor(np.array(y_test)).type(torch.float).to(device)

X_train_norm = torch.Tensor(np.array(X_train_norm)).to(device)
y_train_norm = torch.Tensor(np.array(y_train_norm)).type(torch.float).to(device)
X_test_norm = torch.Tensor(np.array(X_test_norm)).to(device)
y_test_norm = torch.Tensor(np.array(y_test_norm)).type(torch.float).to(device)


In [85]:
print(y_train.shape)

torch.Size([408, 1])


In [105]:
#dataloaders
train = data_utils.TensorDataset(X_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=408, shuffle=True)


test = data_utils.TensorDataset(X_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=175, shuffle=True)

train_norm = data_utils.TensorDataset(X_train_norm, y_train_norm)
train_norm_loader = data_utils.DataLoader(train_norm, batch_size=408, shuffle=True)

test_norm = data_utils.TensorDataset(X_test_norm, y_test_norm)
test_norm_loader = data_utils.DataLoader(test_norm, batch_size=175, shuffle=True)


### 4-5) Model Training

#### 4

In [106]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [140]:
class FullyConnectedNN(nn.Module):

    def __init__(self, hidden_dim):
        super(FullyConnectedNN, self).__init__()
        # layers
        self.linear1 = nn.Linear(10,hidden_dim)
        self.linear2 = nn.Linear(hidden_dim,1)
        #self.linear3 = nn.Linear(hidden_dim, hidden_dim)
        #self.linear4 = nn.Linear(hidden_dim, 1)
        self.activation = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.linear1(x))
        #x = self.activation(self.linear2(x))
        #x = self.activation(self.linear3(x))
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

model = FullyConnectedNN(32)
model.to(device)

loss_BCE = nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)#
print(model)

FullyConnectedNN(
  (linear1): Linear(in_features=10, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
  (activation): ReLU()
  (sigmoid): Sigmoid()
)


In [ ]:
EPOCHS = 500
avg_loss = 0
avg_vloss = 0
f1 = BinaryF1Score()

for epoch in range(EPOCHS+1):
    #train
    model.train()
    for i, train_data in enumerate(train_loader):
        train_inputs, train_labels = train_data
        optimizer.zero_grad()
        predictions = model(train_inputs)
        train_loss = loss_BCE(predictions, train_labels)
        train_loss.backward()
        optimizer.step()

    #validate
    model.eval()

    with torch.no_grad():
        for i, test_data in enumerate(test_loader):
            test_inputs, test_labels = test_data
            test_outputs = model(test_inputs)
            test_loss = loss_BCE(test_outputs, test_labels)
            if epoch == 500:
                print(test_outputs.squeeze())
                print(test_labels.squeeze())
            f1_score =  0 #f1.update(test_outputs.squeeze(), test_labels.squeeze)
            #f1.compute()


    print('EPOCH {}:'.format(epoch))
    print('LOSS train {} valid {} F1 Score{}'.format(train_loss, test_loss,f1_score))

#### 5-a)

The following charts describe the training experiments for different combinations of number of hidden layers and `hidden_size` for `EPOCHS = 500`. The value of the  Binary Cross Entropy loss is displayed for each combination:

* Training:

| hidden size | 1 layer | 2 layers | 3 layers |
|-------------|:-------:|:--------:|----------|
| 32          | 0.4586  |  0.4296  |  0.4558  |
| 64          | 0.4443  |  0.4226  |  0.3316  |
| 128         | 0.4268  |  0.4360  |  0.3665  |
| 256         | 0.4329  |  0.4434  |  0.3401  |

* Validation:

| hidden size | 1 layer | 2 layers | 3 layers |
|-------------|:-------:|:--------:|----------|
| 32          |  0.5085 |  0.5360  |  0.5048  |
| 64          |  0.5247 |  0.5502  |  0.5772  |
| 128         |  0.5287 |  0.5570  |  0.6570  |
| 256         |  0.5479 |  0.7016  |  0.6724  |

We can observe that as the number of layers and the `hidden_size` grow, the training loss tends to have smaller values, but the validation loss tends to grow. This higly suggests that the trained networks are overfitting to the provided dataset. This is mainly caused by the small dataset size and the exagerated overparameterization performed by the networks with more weights. Therefore, using deeper networks is not be the best approach for this problem.

Based on thesse experiments, the chosen architecture was the one with one hidden layer and `hidden_size = 32`:

#### 5-b)

Now we'll utilize the normalized dataset

In [150]:
model = FullyConnectedNN(32)
model.to(device)
loss_BCE = nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)#
print(model)


FullyConnectedNN(
  (linear1): Linear(in_features=10, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
  (activation): ReLU()
  (sigmoid): Sigmoid()
)


In [152]:
EPOCHS = 500
avg_loss = 0
avg_vloss = 0
f1 = BinaryF1Score()

for epoch in range(EPOCHS+1):
    #train
    model.train()
    for i, train_data in enumerate(train_norm_loader):
        train_inputs, train_labels = train_data
        optimizer.zero_grad()
        predictions = model(train_inputs)
        train_loss = loss_BCE(predictions, train_labels)
        train_loss.backward()
        optimizer.step()

    #validate
    model.eval()

    with torch.no_grad():
        for i, test_data in enumerate(test_norm_loader):
            test_inputs, test_labels = test_data
            test_outputs = model(test_inputs)
            test_loss = loss_BCE(test_outputs, test_labels)
            #if epoch == 500:
            #    #print(test_outputs.squeeze())
            #    print(test_labels.squeeze())
            f1_score =  0 #f1.update(test_outputs.squeeze(), test_labels.squeeze)
            #f1.compute()


    print('EPOCH {}:'.format(epoch))
    print('LOSS train {} valid {} F1 Score{}'.format(train_loss, test_loss,f1_score))

EPOCH 0:
LOSS train 0.5062797665596008 valid 0.5195392966270447 F1 Score0
EPOCH 1:
LOSS train 0.5062230825424194 valid 0.5195270776748657 F1 Score0
EPOCH 2:
LOSS train 0.5061663389205933 valid 0.5195087790489197 F1 Score0
EPOCH 3:
LOSS train 0.5061100125312805 valid 0.5194945335388184 F1 Score0
EPOCH 4:
LOSS train 0.5060538649559021 valid 0.519485354423523 F1 Score0
EPOCH 5:
LOSS train 0.5059975385665894 valid 0.5194791555404663 F1 Score0
EPOCH 6:
LOSS train 0.5059411525726318 valid 0.519477367401123 F1 Score0
EPOCH 7:
LOSS train 0.5058844685554504 valid 0.5194758176803589 F1 Score0
EPOCH 8:
LOSS train 0.5058278441429138 valid 0.5194708108901978 F1 Score0
EPOCH 9:
LOSS train 0.5057709217071533 valid 0.5194674134254456 F1 Score0
EPOCH 10:
LOSS train 0.5057135224342346 valid 0.5194582343101501 F1 Score0
EPOCH 11:
LOSS train 0.50565505027771 valid 0.5194392800331116 F1 Score0
EPOCH 12:
LOSS train 0.5055957436561584 valid 0.5194167494773865 F1 Score0
EPOCH 13:
LOSS train 0.5055359601974487

As we can observe, the model actually presented a worse loss for the normalized data. Therefore, we'll keep using the original dataset

#### 5-c)

Now, We'll apply differente learning rates with SGD.
Starting with a high learning rate:

In [177]:
model = FullyConnectedNN(32)
model.to(device)
loss_BCE = nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.1) # torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)#
print(model)

FullyConnectedNN(
  (linear1): Linear(in_features=10, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
  (activation): ReLU()
  (sigmoid): Sigmoid()
)


In [178]:

EPOCHS = 500
avg_loss = 0
avg_vloss = 0
f1 = BinaryF1Score()

for epoch in range(EPOCHS+1):
    #train
    model.train()
    for i, train_data in enumerate(train_norm_loader):
        train_inputs, train_labels = train_data
        optimizer.zero_grad()
        predictions = model(train_inputs)
        train_loss = loss_BCE(predictions, train_labels)
        train_loss.backward()
        optimizer.step()

    #validate
    model.eval()

    with torch.no_grad():
        for i, test_data in enumerate(test_norm_loader):
            test_inputs, test_labels = test_data
            test_outputs = model(test_inputs)
            test_loss = loss_BCE(test_outputs, test_labels)
            #if epoch == 500:
            #    #print(test_outputs.squeeze())
            #    print(test_labels.squeeze())
            f1_score =  0 #f1.update(test_outputs.squeeze(), test_labels.squeeze)
            #f1.compute()


    print('EPOCH {}:'.format(epoch))
    print('LOSS train {} valid {} F1 Score{}'.format(train_loss, test_loss,f1_score))

EPOCH 0:
LOSS train 0.665367066860199 valid 0.6112901568412781 F1 Score0
EPOCH 1:
LOSS train 0.6229644417762756 valid 0.5802013874053955 F1 Score0
EPOCH 2:
LOSS train 0.5913275480270386 valid 0.5761753916740417 F1 Score0
EPOCH 3:
LOSS train 0.5862955451011658 valid 0.5719767808914185 F1 Score0
EPOCH 4:
LOSS train 0.5823042988777161 valid 0.5654455423355103 F1 Score0
EPOCH 5:
LOSS train 0.5768789649009705 valid 0.5593035221099854 F1 Score0
EPOCH 6:
LOSS train 0.5718309283256531 valid 0.5520591735839844 F1 Score0
EPOCH 7:
LOSS train 0.5649257898330688 valid 0.5420025587081909 F1 Score0
EPOCH 8:
LOSS train 0.554789125919342 valid 0.5303467512130737 F1 Score0
EPOCH 9:
LOSS train 0.5434387922286987 valid 0.5206996202468872 F1 Score0
EPOCH 10:
LOSS train 0.5363484621047974 valid 0.5163777470588684 F1 Score0
EPOCH 11:
LOSS train 0.5317609906196594 valid 0.5165848135948181 F1 Score0
EPOCH 12:
LOSS train 0.530439019203186 valid 0.5111492276191711 F1 Score0
EPOCH 13:
LOSS train 0.525336325168609

With a larger learning rate, the model overfitted faster.

In [179]:
model = FullyConnectedNN(32)
model.to(device)
loss_BCE = nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.000001) # torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)#

In [180]:

EPOCHS = 500
avg_loss = 0
avg_vloss = 0
f1 = BinaryF1Score()

for epoch in range(EPOCHS+1):
    #train
    model.train()
    for i, train_data in enumerate(train_norm_loader):
        train_inputs, train_labels = train_data
        optimizer.zero_grad()
        predictions = model(train_inputs)
        train_loss = loss_BCE(predictions, train_labels)
        train_loss.backward()
        optimizer.step()

    #validate
    model.eval()

    with torch.no_grad():
        for i, test_data in enumerate(test_norm_loader):
            test_inputs, test_labels = test_data
            test_outputs = model(test_inputs)
            test_loss = loss_BCE(test_outputs, test_labels)
            #if epoch == 500:
            #    #print(test_outputs.squeeze())
            #    print(test_labels.squeeze())
            f1_score =  0 #f1.update(test_outputs.squeeze(), test_labels.squeeze)
            #f1.compute()


    print('EPOCH {}:'.format(epoch))
    print('LOSS train {} valid {} F1 Score{}'.format(train_loss, test_loss,f1_score))

EPOCH 0:
LOSS train 0.690466582775116 valid 0.6916365027427673 F1 Score0
EPOCH 1:
LOSS train 0.6904647946357727 valid 0.6916345357894897 F1 Score0
EPOCH 2:
LOSS train 0.6904630064964294 valid 0.6916326284408569 F1 Score0
EPOCH 3:
LOSS train 0.6904610991477966 valid 0.6916307806968689 F1 Score0
EPOCH 4:
LOSS train 0.6904593110084534 valid 0.6916288733482361 F1 Score0
EPOCH 5:
LOSS train 0.6904575228691101 valid 0.6916269063949585 F1 Score0
EPOCH 6:
LOSS train 0.6904557347297668 valid 0.6916250586509705 F1 Score0
EPOCH 7:
LOSS train 0.6904539465904236 valid 0.6916231513023376 F1 Score0
EPOCH 8:
LOSS train 0.6904521584510803 valid 0.6916212439537048 F1 Score0
EPOCH 9:
LOSS train 0.6904503703117371 valid 0.6916192770004272 F1 Score0
EPOCH 10:
LOSS train 0.6904485821723938 valid 0.6916174292564392 F1 Score0
EPOCH 11:
LOSS train 0.6904466152191162 valid 0.6916155219078064 F1 Score0
EPOCH 12:
LOSS train 0.6904448866844177 valid 0.6916136145591736 F1 Score0
EPOCH 13:
LOSS train 0.6904430389404

With a very small learning rate, the model didnt converge as fast with the same number of epochs.

Now lets utuilize a LR scheduler:

In [190]:
model = FullyConnectedNN(32)
model.to(device)
loss_BCE = nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)#
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [191]:

EPOCHS = 500
avg_loss = 0
avg_vloss = 0
f1 = BinaryF1Score()

for epoch in range(EPOCHS+1):
    #train
    model.train()
    for i, train_data in enumerate(train_norm_loader):
        train_inputs, train_labels = train_data
        optimizer.zero_grad()
        predictions = model(train_inputs)
        train_loss = loss_BCE(predictions, train_labels)
        train_loss.backward()
        optimizer.step()
    scheduler.step()
    #validate
    model.eval()

    with torch.no_grad():
        for i, test_data in enumerate(test_norm_loader):
            test_inputs, test_labels = test_data
            test_outputs = model(test_inputs)
            test_loss = loss_BCE(test_outputs, test_labels)
            #if epoch == 500:
            #    #print(test_outputs.squeeze())
            #    print(test_labels.squeeze())
            f1_score =  0 #f1.update(test_outputs.squeeze(), test_labels.squeeze)
            #f1.compute()


    print('EPOCH {}:'.format(epoch))
    print('LOSS train {} valid {} F1 Score{}'.format(train_loss, test_loss,f1_score))

EPOCH 0:
LOSS train 0.6930093169212341 valid 0.6918054819107056 F1 Score0
EPOCH 1:
LOSS train 0.6909108757972717 valid 0.6898672580718994 F1 Score0
EPOCH 2:
LOSS train 0.6890451908111572 valid 0.6881396174430847 F1 Score0
EPOCH 3:
LOSS train 0.6873877644538879 valid 0.6866031289100647 F1 Score0
EPOCH 4:
LOSS train 0.6859126687049866 valid 0.6852339506149292 F1 Score0
EPOCH 5:
LOSS train 0.6845986247062683 valid 0.6840113401412964 F1 Score0
EPOCH 6:
LOSS train 0.6834274530410767 valid 0.6829184889793396 F1 Score0
EPOCH 7:
LOSS train 0.682381272315979 valid 0.68194180727005 F1 Score0
EPOCH 8:
LOSS train 0.6814467906951904 valid 0.6810669898986816 F1 Score0
EPOCH 9:
LOSS train 0.680611252784729 valid 0.6802839040756226 F1 Score0
EPOCH 10:
LOSS train 0.6798635721206665 valid 0.6795835494995117 F1 Score0
EPOCH 11:
LOSS train 0.679194450378418 valid 0.678956925868988 F1 Score0
EPOCH 12:
LOSS train 0.6785951852798462 valid 0.6783953309059143 F1 Score0
EPOCH 13:
LOSS train 0.678057849407196 va

We can observe that the combination of a LR scheduler and the Adam optimizer slowed down convergence even more

#### 5-d)

Well first star by utilizing oversample:

In [199]:
grouped = df.groupby(df.Dataset)

In [207]:
positives = grouped.get_group(1)
negatives = grouped.get_group(2)

In [208]:
positives

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
576,32,1,15.0,8.2,289,58,80,5.3,2.2,0.70,1
577,32,1,12.7,8.4,190,28,47,5.4,2.6,0.90,1
579,40,1,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,1,0.8,0.2,245,48,49,6.4,3.2,1.00,1


In [210]:
negatives

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
8,17,1,0.9,0.3,202,22,19,7.4,4.1,1.20,2
12,64,1,0.9,0.3,310,61,58,7.0,3.4,0.90,2
15,25,1,0.6,0.1,183,91,53,5.5,2.3,0.70,2
17,33,1,1.6,0.5,165,15,23,7.3,3.5,0.92,2
24,63,1,0.9,0.2,194,52,45,6.0,3.9,1.85,2
...,...,...,...,...,...,...,...,...,...,...,...
551,29,1,1.2,0.4,160,20,22,6.2,3.0,0.90,2
564,38,0,0.6,0.1,165,22,34,5.9,2.9,0.90,2
566,50,0,1.0,0.3,191,22,31,7.8,4.0,1.00,2
578,60,1,0.5,0.1,500,20,34,5.9,1.6,0.37,2


we can se the positives and negatives are higly unbalanced

In [216]:
negatives_upsampled = pd.concat([negatives,negatives,negatives])

In [217]:
negatives_upsampled

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
8,17,1,0.9,0.3,202,22,19,7.4,4.1,1.20,2
12,64,1,0.9,0.3,310,61,58,7.0,3.4,0.90,2
15,25,1,0.6,0.1,183,91,53,5.5,2.3,0.70,2
17,33,1,1.6,0.5,165,15,23,7.3,3.5,0.92,2
24,63,1,0.9,0.2,194,52,45,6.0,3.9,1.85,2
...,...,...,...,...,...,...,...,...,...,...,...
551,29,1,1.2,0.4,160,20,22,6.2,3.0,0.90,2
564,38,0,0.6,0.1,165,22,34,5.9,2.9,0.90,2
566,50,0,1.0,0.3,191,22,31,7.8,4.0,1.00,2
578,60,1,0.5,0.1,500,20,34,5.9,1.6,0.37,2


Now the new dataset conttains 501 negative and 416 poasitive cacses, almost 50/50

In [219]:
df_upsampled = pd.concat([positives,negatives_upsampled])
df_upsampled

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
551,29,1,1.2,0.4,160,20,22,6.2,3.0,0.90,2
564,38,0,0.6,0.1,165,22,34,5.9,2.9,0.90,2
566,50,0,1.0,0.3,191,22,31,7.8,4.0,1.00,2
578,60,1,0.5,0.1,500,20,34,5.9,1.6,0.37,2


In [252]:
df_norm_up = (df_upsampled-df_upsampled.min())/(df_upsampled.max()-df_upsampled.min())
X_train_up, X_test_up, y_train_up, y_test_up = sklearn.model_selection.train_test_split(df_upsampled.iloc[:,:-1], df_norm_up.iloc[:,-1:], test_size= 0.3, random_state=1)


#df to tensor
X_train_up = torch.Tensor(np.array(X_train_up)).to(device)
y_train_up = torch.Tensor(np.array(y_train_up)).type(torch.float).to(device)
X_test_up = torch.Tensor(np.array(X_test_up)).to(device)
y_test_up = torch.Tensor(np.array(y_test_up)).type(torch.float).to(device)
print(y_train_up.shape)
print(y_test_up.shape)

torch.Size([641, 1])
torch.Size([276, 1])


In [258]:
train_up = data_utils.TensorDataset(X_train_up, y_train_up)
train_loader_up = data_utils.DataLoader(train_up, batch_size=641, shuffle=True)


test_up = data_utils.TensorDataset(X_test_up, y_test_up)
test_loader_up = data_utils.DataLoader(test_up, batch_size=276, shuffle=True)

Now lets train with the upsampled dataset

In [265]:
model = FullyConnectedNN(32)
model.to(device)
loss_BCE = nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)#

In [266]:
EPOCHS = 500
avg_loss = 0
avg_vloss = 0
f1 = BinaryF1Score()

for epoch in range(EPOCHS+1):
    #train
    model.train()
    for i, train_data in enumerate(train_loader_up):
        train_inputs, train_labels = train_data
        optimizer.zero_grad()
        predictions = model(train_inputs)
        train_loss = loss_BCE(predictions, train_labels)
        train_loss.backward()
        optimizer.step()

    #validate
    model.eval()

    with torch.no_grad():
        for i, test_data in enumerate(test_loader_up):
            test_inputs, test_labels = test_data
            test_outputs = model(test_inputs)
            test_loss = loss_BCE(test_outputs, test_labels)
            #if epoch == 500:
                #print(test_outputs.squeeze())
                #print(test_labels.squeeze())
            f1_score =  0 #f1.update(test_outputs.squeeze(), test_labels.squeeze)
            #f1.compute()


    print('EPOCH {}:'.format(epoch))
    print('LOSS train {} valid {} F1 Score{}'.format(train_loss, test_loss,f1_score))

EPOCH 0:
LOSS train 1.77676522731781 valid 1.1345199346542358 F1 Score0
EPOCH 1:
LOSS train 1.270626187324524 valid 1.0860153436660767 F1 Score0
EPOCH 2:
LOSS train 1.2642635107040405 valid 1.0408382415771484 F1 Score0
EPOCH 3:
LOSS train 1.2193965911865234 valid 0.9676786065101624 F1 Score0
EPOCH 4:
LOSS train 1.1404894590377808 valid 0.9116004109382629 F1 Score0
EPOCH 5:
LOSS train 1.0782756805419922 valid 0.8896906971931458 F1 Score0
EPOCH 6:
LOSS train 1.0526928901672363 valid 0.8682500123977661 F1 Score0
EPOCH 7:
LOSS train 1.035385012626648 valid 0.8307100534439087 F1 Score0
EPOCH 8:
LOSS train 1.0018434524536133 valid 0.8083972930908203 F1 Score0
EPOCH 9:
LOSS train 0.9740129113197327 valid 0.8160683512687683 F1 Score0
EPOCH 10:
LOSS train 0.9749239683151245 valid 0.8251928687095642 F1 Score0
EPOCH 11:
LOSS train 0.9819231033325195 valid 0.8186995983123779 F1 Score0
EPOCH 12:
LOSS train 0.977398693561554 valid 0.8047881722450256 F1 Score0
EPOCH 13:
LOSS train 0.9688634872436523 

Now well apply downsampling to balance the dataset:

In [261]:
positives_down = positives.iloc[0:167].copy()
df_downsampled = pd.concat([positives_down,negatives])
df_downsampled

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
551,29,1,1.2,0.4,160,20,22,6.2,3.0,0.90,2
564,38,0,0.6,0.1,165,22,34,5.9,2.9,0.90,2
566,50,0,1.0,0.3,191,22,31,7.8,4.0,1.00,2
578,60,1,0.5,0.1,500,20,34,5.9,1.6,0.37,2


datasets are balanced 50/50 by downsampling. Niow lets perform training

In [262]:
df_norm_down = (df_downsampled-df_downsampled.min())/(df_downsampled.max()-df_downsampled.min())
X_train_down, X_test_down, y_train_down, y_test_down = sklearn.model_selection.train_test_split(df_downsampled.iloc[:,:-1], df_norm_down.iloc[:,-1:], test_size= 0.3, random_state=1)


#df to tensor
X_train_down = torch.Tensor(np.array(X_train_down)).to(device)
y_train_down = torch.Tensor(np.array(y_train_down)).type(torch.float).to(device)
X_test_down = torch.Tensor(np.array(X_test_down)).to(device)
y_test_down = torch.Tensor(np.array(y_test_down)).type(torch.float).to(device)
print(y_train_down.shape)
print(y_test_down.shape)


torch.Size([233, 1])
torch.Size([101, 1])


In [263]:
train_down = data_utils.TensorDataset(X_train_down, y_train_down)
train_loader_down = data_utils.DataLoader(train_down, batch_size=233, shuffle=True)


test_down = data_utils.TensorDataset(X_test_down, y_test_down)
test_loader_down = data_utils.DataLoader(test_down, batch_size=101, shuffle=True)

In [264]:
model = FullyConnectedNN(32)
model.to(device)
loss_BCE = nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)#

In [267]:
EPOCHS = 500
avg_loss = 0
avg_vloss = 0
f1 = BinaryF1Score()

for epoch in range(EPOCHS+1):
    #train
    model.train()
    for i, train_data in enumerate(train_loader_down):
        train_inputs, train_labels = train_data
        optimizer.zero_grad()
        predictions = model(train_inputs)
        train_loss = loss_BCE(predictions, train_labels)
        train_loss.backward()
        optimizer.step()

    #validate
    model.eval()

    with torch.no_grad():
        for i, test_data in enumerate(test_loader_down):
            test_inputs, test_labels = test_data
            test_outputs = model(test_inputs)
            test_loss = loss_BCE(test_outputs, test_labels)
            #if epoch == 500:
                #print(test_outputs.squeeze())
                #print(test_labels.squeeze())
            f1_score =  0 #f1.update(test_outputs.squeeze(), test_labels.squeeze)
            #f1.compute()


    print('EPOCH {}:'.format(epoch))
    print('LOSS train {} valid {} F1 Score{}'.format(train_loss, test_loss,f1_score))

EPOCH 0:
LOSS train 0.508406937122345 valid 0.5028596520423889 F1 Score0
EPOCH 1:
LOSS train 0.5082763433456421 valid 0.5001372694969177 F1 Score0
EPOCH 2:
LOSS train 0.5061447620391846 valid 0.49997180700302124 F1 Score0
EPOCH 3:
LOSS train 0.5057663321495056 valid 0.4981409013271332 F1 Score0
EPOCH 4:
LOSS train 0.5027737021446228 valid 0.5012329816818237 F1 Score0
EPOCH 5:
LOSS train 0.5036921501159668 valid 0.4986477494239807 F1 Score0
EPOCH 6:
LOSS train 0.5003344416618347 valid 0.504458487033844 F1 Score0
EPOCH 7:
LOSS train 0.5011932253837585 valid 0.4995187222957611 F1 Score0
EPOCH 8:
LOSS train 0.49743416905403137 valid 0.5040194988250732 F1 Score0
EPOCH 9:
LOSS train 0.497991144657135 valid 0.5020749568939209 F1 Score0
EPOCH 10:
LOSS train 0.49483323097229004 valid 0.5051590800285339 F1 Score0
EPOCH 11:
LOSS train 0.4952027499675751 valid 0.5055606365203857 F1 Score0
EPOCH 12:
LOSS train 0.492264986038208 valid 0.5119428634643555 F1 Score0
EPOCH 13:
LOSS train 0.4921141564846

We can observea worseperformance in the loss representation. Neverthelles, by analyzing accuracy and the F1 scores we should see a better performance.

5-e)

I faced A problem with my models classification function. The model stopped converging in a situation where most of the outputs were very close to 0. Therefore, even though the loss converged to a minima, the model performance was still not good enough.

For this reason, I wasn able to complete this question.